In [1]:
import tensorflow as tf
import numpy as np

tf.enable_eager_execution()

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


position encoding + [convolution-layer × # + self-attention-layer + feed-forward-layer]

> We use depthwise separable convolutions (Chollet, 2016) (Kaiser et al., 2017) rather than traditional ones, as we observe that it is memory efficient and has better generalization.

これはkerasが持ってくれている - [SeparableConv2D](https://keras.io/layers/convolutional/#separableconv2d)

> For the self-attention-layer, we adopt the multi-head attention mechanism

これは自前実装が必要そう

勉強だから自分で実装するけど、答えは[ここ](https://github.com/tensorflow/tensor2tensor/blob/master/tensor2tensor/layers/common_attention.py#L2557)

> A positional encoding is added to the input at the beginning of each encoder layer consisting of sin and cos functions at varying wavelengths, as defined in (Vaswani et al., 2017a). 

これも自前かな？元論文にあたる

tensor2tensorに[実装](https://github.com/tensorflow/tensor2tensor/blob/ed9e3bdfd0292d4b6e5b1a1bf272146c8e2f5e9f/tensor2tensor/layers/common_attention.py#L386)がある。これ使わせてもらいたいなぁ

tensor2tensor自体をpip installしてimportできちゃった、超横着

(これやりだすとmulti-head attentionだってimportできちゃうんじゃないのか？勉強したいから自分で書くつもりだけど…)

> Each of these basic operations (conv/self-attention/ffn) is placed inside a residual block

これも自前かな？

ドロップアウト系は後で見直しが必要だな

layernormとかresidual networkとかはtransformerでもやっている

In [2]:
import tensor2tensor.layers.common_attention

In [3]:
tensor2tensor.layers.common_attention.get_timing_signal_1d?

In [4]:
# Scaled Dot-Product Attention

d_k = 20
d_v = 24
n = 5
batch_size = 2

def dot_product_attention(Q, K, V):
    d_k = tf.cast(K.shape[-1], tf.float32)
    return tf.matmul(tf.nn.softmax(
        tf.matmul(Q, K, transpose_b=True) / tf.sqrt(d_k)), V)

In [6]:
np.random.seed(1234)

Q = np.random.normal(0, 1e-3, size=(batch_size, n, d_k)).astype(np.float32)
# K = np.random.randn(batch_size, n, d_k).astype(np.float32)
# V = np.random.randn(batch_size, n, d_v).astype(np.float32)
print(Q)
dot_product_attention(Q, Q, Q)

[[[ 4.7143517e-04 -1.1909757e-03  1.4327070e-03 -3.1265189e-04
   -7.2058872e-04  8.8716293e-04  8.5958838e-04 -6.3652353e-04
    1.5696372e-05 -2.2426851e-03  1.1500358e-03  9.9194597e-04
    9.5332414e-04 -2.0212547e-03 -3.3407737e-04  2.1183646e-06
    4.0545341e-04  2.8909193e-04  1.3211581e-03 -1.5469056e-03]
  [-2.0264632e-04 -6.5596937e-04  1.9342138e-04  5.5343891e-04
    1.3181516e-03 -4.6930529e-04  6.7555410e-04 -1.8170272e-03
   -1.8310854e-04  1.0589692e-03 -3.9784022e-04  3.3743764e-04
    1.0475785e-03  1.0459382e-03  8.6371729e-04 -1.2209157e-04
    1.2471295e-04 -3.2279480e-04  8.4167469e-04  2.3909605e-03]
  [ 7.6199591e-05 -5.6644593e-04  3.6141937e-05 -2.0749776e-03
    2.4779220e-04 -8.9715677e-04 -1.3679483e-04  1.8289191e-05
    7.5541396e-04  2.1526858e-04  8.4100879e-04 -1.4458101e-03
   -1.4019733e-03 -1.0091820e-04 -5.4824247e-04 -1.4461951e-04
    3.5402033e-04 -3.5513025e-05  5.6573830e-04  1.5456588e-03]
  [-9.7423635e-04 -7.0344875e-05  3.0796885e-04 -2.0

<tf.Tensor: id=42, shape=(2, 5, 20), dtype=float32, numpy=
array([[[-3.2970453e-05, -1.2094510e-03,  6.5826997e-04, -3.7801167e-04,
          4.0873562e-04, -6.6196796e-04,  8.3926407e-04, -5.1859429e-04,
          2.1414403e-04,  2.2564919e-04,  1.7752343e-04, -1.0865182e-05,
          5.5062141e-05, -2.2749124e-04, -2.5652911e-05,  3.3226027e-04,
          1.5327682e-04, -1.3871676e-04,  4.5831295e-04,  3.8346634e-04],
        [-3.2971722e-05, -1.2094495e-03,  6.5826817e-04, -3.7801109e-04,
          4.0873824e-04, -6.6197052e-04,  8.3926419e-04, -5.1859574e-04,
          2.1414395e-04,  2.2565328e-04,  1.7752127e-04, -1.0866135e-05,
          5.5062108e-05, -2.2748725e-04, -2.5652200e-05,  3.3226097e-04,
          1.5327659e-04, -1.3871798e-04,  4.5831254e-04,  3.8347094e-04],
        [-3.2971122e-05, -1.2094503e-03,  6.5826846e-04, -3.7801272e-04,
          4.0873716e-04, -6.6197006e-04,  8.3926338e-04, -5.1859417e-04,
          2.1414456e-04,  2.2565243e-04,  1.7752245e-04, -1.086

In [7]:
np.random.seed(1234)

Q = np.random.normal(0, 1e-3, size=(batch_size, n, d_k)).astype(np.float64)

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)

x = np.dot(Q[0], Q[0].T) * (20**-0.5)
x

array([[ 5.15863321e-06, -1.20151391e-06, -6.91062336e-07,
        -4.09958150e-07,  3.88991460e-07],
       [-1.20151391e-06,  3.90307129e-06,  2.90966772e-07,
         1.91580645e-06, -7.18850603e-08],
       [-6.91062336e-07,  2.90966772e-07,  3.14534453e-06,
         2.72977704e-07,  6.26483062e-07],
       [-4.09958150e-07,  1.91580645e-06,  2.72977704e-07,
         4.95617454e-06, -1.64216258e-07],
       [ 3.88991460e-07, -7.18850603e-08,  6.26483062e-07,
        -1.64216258e-07,  5.03246109e-06]])

In [47]:
dot_product_attention(Q, Q, Q)

<tf.Tensor: id=815, shape=(2, 5, 20), dtype=float32, numpy=
array([[[ 0.46368584, -1.2038844 ,  1.421492  , -0.3118993 ,
         -0.70041174,  0.8564762 ,  0.8600776 , -0.6250482 ,
          0.0203185 , -2.1887264 ,  1.1303378 ,  0.97032714,
          0.9293498 , -1.9890662 , -0.32527012,  0.00831646,
          0.397902  ,  0.28371736,  1.2956773 , -1.5168614 ],
        [-0.2712941 , -0.6350787 ,  0.22719535,  0.39516327,
          1.2324362 , -0.6941183 ,  0.8157225 , -1.6473185 ,
         -0.10807492,  0.9873781 , -0.39895055,  0.3030086 ,
          0.91911054,  0.91825277,  0.61843365,  0.12422507,
          0.13572499, -0.39537975,  0.78350025,  2.0347147 ],
        [ 0.04686259, -0.7571781 ,  0.16609944, -1.6849325 ,
          0.31234026, -0.8842332 ,  0.08002034, -0.0702003 ,
          0.64048505,  0.31232   ,  0.66177857, -1.1601456 ,
         -1.1214842 , -0.08367237, -0.40477702, -0.03669615,
          0.29188663, -0.05797445,  0.48701817,  1.3245255 ],
        [-0.91620606, 

In [48]:
n_heads = 3

np.random.seed(1234)

Q = np.random.randn(batch_size, n_heads, n, d_k).astype(np.float32)
K = np.random.randn(batch_size, n_heads, n, d_k).astype(np.float32)
V = np.random.randn(batch_size, n_heads, n, d_v).astype(np.float32)

dot_product_attention(Q, K, V)

<tf.Tensor: id=835, shape=(2, 3, 5, 24), dtype=float32, numpy=
array([[[[-9.10805404e-01,  4.68819849e-02,  1.45135999e+00,
           5.59234321e-01, -7.83527792e-01, -2.24587411e-01,
           2.45982647e-01,  5.36981404e-01,  9.59708214e-01,
          -9.08062935e-01, -7.09595859e-01,  4.49533701e-01,
           3.50889385e-01, -1.28543049e-01,  1.96838185e-01,
           2.00286180e-01, -1.26194596e-01,  8.00544679e-01,
          -2.02491522e-01,  2.73273259e-01, -9.03197050e-01,
          -7.54610717e-01,  5.20642638e-01, -5.14845669e-01],
         [ 1.46483660e-01,  1.55516803e+00, -1.37622738e+00,
          -9.71389890e-01,  7.42058575e-01, -3.84970754e-01,
           2.06635332e+00, -4.40845370e-01,  5.81134677e-01,
          -1.17497683e+00,  5.92545092e-01,  2.40105629e+00,
           4.31423783e-01,  1.67123652e+00,  9.21944916e-01,
          -3.88504177e-01, -1.65141046e+00,  1.36039245e+00,
           4.98294473e-01, -3.44438255e-01, -9.86966789e-01,
           5.46279430

In [49]:
dot_product_attention(Q, Q, Q)

<tf.Tensor: id=855, shape=(2, 3, 5, 20), dtype=float32, numpy=
array([[[[ 4.63685840e-01, -1.20388436e+00,  1.42149198e+00,
          -3.11899304e-01, -7.00411737e-01,  8.56476188e-01,
           8.60077620e-01, -6.25048220e-01,  2.03185007e-02,
          -2.18872643e+00,  1.13033783e+00,  9.70327139e-01,
           9.29349780e-01, -1.98906624e+00, -3.25270116e-01,
           8.31645541e-03,  3.97902012e-01,  2.83717364e-01,
           1.29567730e+00, -1.51686144e+00],
         [-2.71294087e-01, -6.35078728e-01,  2.27195352e-01,
           3.95163268e-01,  1.23243618e+00, -6.94118321e-01,
           8.15722525e-01, -1.64731848e+00, -1.08074918e-01,
           9.87378120e-01, -3.98950547e-01,  3.03008586e-01,
           9.19110537e-01,  9.18252766e-01,  6.18433654e-01,
           1.24225073e-01,  1.35724992e-01, -3.95379752e-01,
           7.83500254e-01,  2.03471470e+00],
         [ 4.68625873e-02, -7.57178128e-01,  1.66099444e-01,
          -1.68493247e+00,  3.12340260e-01, -8.8423317

In [7]:
np.random.seed(1234)

d_model = 20

Q = np.random.randn(batch_size, n, d_model)
W_Q = np.random.randn(d_model, d_k)

print(Q.shape, W_Q.shape)

(2, 5, 20) (20, 20)


In [8]:
[Q.shape[-1]]

[20]

In [9]:
np.dot(Q[0], W_Q)

array([[-6.02186368e+00,  4.71036170e+00,  4.78394407e-01,
        -1.95374941e+00, -3.77729152e+00, -9.03798581e-01,
         9.29919941e+00,  4.07358160e+00,  6.91877445e+00,
        -1.03789112e+00, -5.33030335e+00,  2.94333103e-03,
         8.36836844e-01, -2.43775123e+00,  1.69049897e+00,
        -9.72307875e-01, -1.01871267e+00,  5.45488202e+00,
        -2.25399293e+00, -1.11217924e+01],
       [-8.12207869e-01, -4.47643595e+00, -2.39715340e+00,
         2.19649059e+00, -7.79263434e-01,  8.36597112e-01,
         9.88349936e-01, -1.51310167e-01, -6.97845228e+00,
         3.78534711e-01,  3.59889430e+00, -3.08007472e+00,
         2.68039262e+00, -1.01589415e+00, -1.61473832e+00,
         3.14471846e+00, -5.26927758e+00, -8.46149372e+00,
        -2.40889099e+00,  4.95341156e+00],
       [-2.29957305e+00, -2.13494800e+00,  1.07428170e+00,
        -6.38390754e+00,  3.69538366e+00,  4.92677788e-01,
        -3.51141490e+00, -4.40223549e+00,  4.43084002e+00,
         3.17779349e+00,  2.9

In [10]:
assert all([all(np.isclose(x, y)) for x, y in zip(
    tf.tensordot(Q, W_Q, [[len(Q.shape) - 1], [0]])[0],
    np.dot(Q[0], W_Q))])

In [11]:
assert all([all(np.isclose(x, y)) for x, y in zip(
    tf.tensordot(Q, W_Q, [[len(Q.shape) - 1], [0]])[1],
    np.dot(Q[1], W_Q))])

In [50]:
def split_heads(x, num_heads):
    """xの最後の次元をnum_headsに分ける

    引数:
      x: (batch_size, N, num_heads * dim)
    戻り値: (batch_size, num_heads, N, dim)
    """
    shape = x.shape.as_list()
    # (batch_size, N, num_heads, dim)
    splitted = tf.reshape(
        x, shape[:-1] + [num_heads, shape[-1] // num_heads])
    # (batch_size, num_heads, N, dim)
    return tf.transpose(splitted, [0, 2, 1, 3])

def combine_heads(x):
    """xの最後の次元をnum_heads * d_vに戻す
    
    引数:
      x: (batch_size, num_heads, N, d_v)
    戻り値: (batch_size, N, num_heads * d_v)
    """
    x = tf.transpose(x, [0, 2, 1, 3])
    shape = x.shape.as_list()
    return tf.reshape(x, shape[:-2] + [shape[-2] * shape[-1]])

class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, num_heads, d_k, d_v, **kwargs):
        self._num_heads = num_heads
        self._d_k = d_k
        self._d_v = d_v
        super(MultiHeadAttention, self).__init__(**kwargs)

    def build(self, input_shape):
        Q_shape, _, _ = input_shape
        d_model = Q_shape[-1]
        
        self._W_Q = self.add_weight(
            'W_Q',
            [d_model, self._num_heads * self._d_k],
            initializer='glorot_uniform')
        self._W_K = self.add_weight(
            'W_K',
            [d_model, self._num_heads * self._d_k],
            initializer='glorot_uniform')
        self._W_V = self.add_weight(
            'W_V',
            [d_model, self._num_heads * self._d_v],
            initializer='glorot_uniform')
        self._W_O = self.add_weight(
            'W_O',
            [self._num_heads * self._d_v, d_model],
            initializer='glorot_uniform')
        
        return super(MultiHeadAttention, self).build(input_shape)
    
    def call(self, x):
        # (batch_size, N, d_model)
        q, k, v = x
        
        # (batch_size, N, num_heads * d_k)
        q_W_Q = tf.tensordot(q, self._W_Q, [[2], [0]])
        k_W_K = tf.tensordot(k, self._W_K, [[2], [0]])
        # (batch_size, N, num_heads * d_v)
        v_W_V = tf.tensordot(v, self._W_V, [[2], [0]])
        # (batch_size, num_heads, N, d_k)
        q_W_Q = split_heads(q_W_Q, self._num_heads)
        k_W_K = split_heads(k_W_K, self._num_heads)
        # (batch_size, num_heads, N, d_v)
        v_W_V = split_heads(v_W_V, self._num_heads)
        
        # (batch_size, num_heads, N, d_v)
        x = dot_product_attention(q_W_Q, k_W_K, v_W_V)
        # (batch_size, N, num_heads * d_v)
        x = combine_heads(x)

        # (batch_size, N, d_model)
        return tf.tensordot(x, self._W_O, [[2], [0]])
    
    def compute_output_shape(self, input_shape):
        return input_shape[0]

In [51]:
d_k = 20
d_v = 24
d_model = 12
n = 5

inputs = [tf.keras.layers.Input(shape=(n, d_model,)) for _ in range(3)]
multi_head_attention = MultiHeadAttention(8, d_k, d_v)(inputs)

model = tf.keras.models.Model(
    inputs=inputs, outputs=multi_head_attention)
model.compile(
    optimizer=tf.train.GradientDescentOptimizer(0.001),
    loss='mse')

In [55]:
Q, K, V = [np.random.normal(scale=1e-4, size=(2, n, d_model)).astype(np.float32) for _ in range(3)]

print(Q)

model.predict([Q, Q, Q])

[[[-7.74300061e-05 -1.86986610e-04  8.12736907e-05  6.47766792e-05
    4.98688569e-05  1.13713337e-04 -1.68398378e-06  5.88231233e-05
    1.24583612e-05 -9.44013391e-06  9.53379858e-05  8.37739426e-05]
  [ 2.56399246e-04 -1.30790795e-04  7.81865674e-05 -6.44111133e-05
   -9.24558481e-05 -5.84338432e-06 -1.23936072e-04 -1.06322681e-04
    1.78832372e-04  1.37649215e-04  1.78669099e-04  7.95562573e-06]
  [ 1.19301389e-04  7.84929762e-06  1.37047886e-04 -7.00814417e-05
   -8.72384335e-05  4.87689031e-05  5.84534391e-05 -1.75387293e-04
    6.45651162e-05  1.55177579e-04 -1.22476325e-04 -1.22126934e-04]
  [ 6.03332883e-05  1.59796964e-05  9.73776405e-05 -9.57086522e-05
   -1.39408250e-04 -9.70552501e-05 -9.48372908e-05 -4.96041539e-05
    4.15224131e-05 -3.17636432e-06 -5.89992997e-05  6.80320081e-05]
  [-3.71570422e-05 -1.34657035e-04 -9.96609815e-05  8.35854444e-05
   -1.20334676e-04  4.66394886e-05 -1.09040076e-04 -1.02196267e-04
    5.71628261e-05  1.05248102e-04  1.78473216e-04  1.3645

array([[[-1.40354668e-05, -4.38052011e-05,  7.95058986e-06,
         -1.65997517e-05,  7.98833389e-06, -7.83571595e-05,
          5.95658639e-06, -2.99418825e-05,  2.19081230e-05,
         -2.08398997e-05,  1.10648780e-05,  7.87070530e-06],
        [-1.40354668e-05, -4.38052011e-05,  7.95058986e-06,
         -1.65997517e-05,  7.98833389e-06, -7.83571595e-05,
          5.95658639e-06, -2.99418825e-05,  2.19081230e-05,
         -2.08398997e-05,  1.10648780e-05,  7.87070530e-06],
        [-1.40354668e-05, -4.38052011e-05,  7.95058986e-06,
         -1.65997517e-05,  7.98833389e-06, -7.83571595e-05,
          5.95658639e-06, -2.99418825e-05,  2.19081230e-05,
         -2.08398997e-05,  1.10648780e-05,  7.87070530e-06],
        [-1.40354668e-05, -4.38052011e-05,  7.95058986e-06,
         -1.65997517e-05,  7.98833389e-06, -7.83571595e-05,
          5.95658639e-06, -2.99418825e-05,  2.19081230e-05,
         -2.08398997e-05,  1.10648780e-05,  7.87070530e-06],
        [-1.40354668e-05, -4.3805201

In [64]:
batch_size = 2
N = 10 # number of maximum context length
M = 5 # number of maximum question length
p1 = 300 # word embedding size
C = 16 # number of maximum word length
char_vocab_size = 1000
p2 = 200 # character embedding size
filter_size = 7
num_heads = 8
num_embedding_encoder_conv_layers = 4
d = 128

inputs = tf.keras.Input(shape=(N, p1 + p2))

# (batch_size, 1, N, p1 + p2)
x = tf.keras.layers.Reshape((1, N, p1 + p2))(inputs)
print(x)

for l in range(num_embedding_encoder_conv_layers):
    # (batch_size, 1, N, 128)
    print(x)
    x = tf.keras.layers.SeparableConv2D(
        d,
        kernel_size=(1, 7), padding='same')(x)

model = tf.keras.models.Model(
    inputs=inputs, outputs=x)
model.compile(
    optimizer=tf.train.GradientDescentOptimizer(0.001),
    loss='mse')

DeferredTensor('None', shape=(?, 1, 10, 500), dtype=float32)
DeferredTensor('None', shape=(?, 1, 10, 500), dtype=float32)
DeferredTensor('None', shape=(?, 1, 10, 128), dtype=float32)
DeferredTensor('None', shape=(?, 1, 10, 128), dtype=float32)
DeferredTensor('None', shape=(?, 1, 10, 128), dtype=float32)


In [65]:
from tensor2tensor.layers.common_attention import add_timing_signal_1d

class PositionEncoding(tf.keras.layers.Lambda):
    def __init__(self, **kwargs):
        super(PositionEncoding, self).__init__(
            function=lambda x: add_timing_signal_1d(x),
            **kwargs)
    
    def compute_output_shape(self, input_shape):
        return input_shape

inputs = tf.keras.Input(shape=(N, p1 + p2))
x = PositionEncoding()(inputs)
# (batch_size, 1, N, p1 + p2)
x = tf.keras.layers.Reshape((1, N, p1 + p2))(x)

for l in range(num_embedding_encoder_conv_layers):
    # (batch_size, 1, N, d)
    x = tf.keras.layers.SeparableConv2D(
        d,
        kernel_size=(1, 7),
        padding='same')(x)
    x = tf.keras.layers.Activation('relu')(x)

# (batch_size, N, d)
x = tf.keras.layers.Reshape((N, -1))(x)
# (batch_size, N, d)
x = MultiHeadAttention(num_heads, d, d)([x] * 3)
# (batch_size, N, d)
x = tf.keras.layers.Dense(d)(x)
x = tf.keras.layers.Activation('relu')(x)
    
model = tf.keras.models.Model(
    inputs=inputs, outputs=x)
model.compile(
    optimizer=tf.train.GradientDescentOptimizer(0.001),
    loss='mse')

In [66]:
model.predict(np.random.randn(batch_size, N, p1 + p2)).shape

(2, 10, 128)

一応出来たけど不安…少なくともdropoutは後で追加する、後layer normalization、layer dropout

NLPLearn/QANetだとmultihead attentionに畳込みを使っているのが気になる。
QANetにもTransformerにも元論文にそんな記述は見つけていない。

ただし、transformerには畳込みの実装もある。